# Chaining Experiments

Setting up langchain imports.

In [17]:
from langchain_openai import OpenAI;
from langchain.prompts import PromptTemplate;
from langchain_core.output_parsers import StrOutputParser;
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnablePick;
import json;
from dotenv import load_dotenv;
load_dotenv();

Using OpenAI

In [18]:
llm = OpenAI();

Code Chain
```mermaid
graph TD;
    language & task-->code_chain;
    code_chain-->code;
```

In [19]:
code_prompt = PromptTemplate.from_template("Write a very short {language} function that will {task}");
code_chain = code_prompt | llm | {"code": StrOutputParser()};

print(code_chain.input_schema.schema());
print(code_chain.output_schema.schema());

{'title': 'PromptInput', 'type': 'object', 'properties': {'language': {'title': 'Language', 'type': 'string'}, 'task': {'title': 'Task', 'type': 'string'}}}
{'title': 'RunnableParallel<code>Output', 'type': 'object', 'properties': {'code': {'title': 'Code', 'type': 'string'}}}


Test Chain

```mermaid
graph TD;
    language & code-->test_chain;
    test_chain-->test
```

In [20]:
test_prompt = PromptTemplate.from_template("Write test code for the following {language} code:\n{code}");
test_chain = test_prompt | llm | {"test": StrOutputParser()};

print(test_chain.input_schema.schema());
print(test_chain.output_schema.schema());

{'title': 'PromptInput', 'type': 'object', 'properties': {'code': {'title': 'Code', 'type': 'string'}, 'language': {'title': 'Language', 'type': 'string'}}}
{'title': 'RunnableParallel<test>Output', 'type': 'object', 'properties': {'test': {'title': 'Test', 'type': 'string'}}}


Chaining Code and Test together

```mermaid
graph TD;
    language & task-->code_chain;
    code_chain-->code;
    language & code-->test_chain;
    test_chain-->test
```

To maintain the inputs in the pipeline for subsequent steps we can use `RunnableParallel`, to fully control the flow of variables through the pipeline.

I did not know that an asked this question on stackoverflow: [How to pass input parameters through RunnableSequences in LangChain v0.2?](https://stackoverflow.com/questions/78503869/how-to-pass-input-parameters-through-runnablesequences-in-langchain-v0-2)
The first responder gave a non-working answer, but put me on the right path by mentioning `LCEL` (LangChain Expression Language). So I found this question [How to include the inputs of the first chain to the second chain in LangChain's SequentialChain?](https://stackoverflow.com/a/77525478/6466378).

In [21]:
chain = RunnableParallel({
    "language": RunnablePick("language"),
    "code": code_chain | RunnablePick("code"),
}) | RunnableParallel({
    "language": RunnablePick("language"),
    "code": RunnablePick("code"),
    "test": test_chain | RunnablePick("test"),
});

print(chain.input_schema.schema())
print(chain.output_schema.schema())

{'title': 'RunnableParallel<language,code>Input', 'type': 'object', 'properties': {'language': {'title': 'Language', 'type': 'string'}, 'task': {'title': 'Task', 'type': 'string'}}}
{'title': 'RunnableParallel<language,code,test>Output', 'type': 'object', 'properties': {'language': {'title': 'Language', 'type': 'object'}, 'code': {'title': 'Code', 'type': 'object'}, 'test': {'title': 'Test', 'type': 'object'}}}


In [22]:
result = chain.invoke({
    "language": "python",
    "task": "reverse a string",
});

print(json.dumps(result, indent=4));

{
    "language": "python",
    "code": "\n\ndef reverse_string(string):\n    return string[::-1]",
    "test": "\n\n# Importing the necessary libraries\nimport unittest\n\n# Creating a class for testing the reverse_string function\nclass TestReverseString(unittest.TestCase):\n    \n    # Creating a test case for a string with even length\n    def test_even_length_string(self):\n        self.assertEqual(reverse_string(\"Hello\"), \"olleH\")\n    \n    # Creating a test case for a string with odd length\n    def test_odd_length_string(self):\n        self.assertEqual(reverse_string(\"Python\"), \"nohtyP\")\n    \n    # Creating a test case for an empty string\n    def test_empty_string(self):\n        self.assertEqual(reverse_string(\"\"), \"\")\n    \n    # Creating a test case for a string with only one character\n    def test_single_character_string(self):\n        self.assertEqual(reverse_string(\"a\"), \"a\")\n        \n    # Creating a test case for a string with special character